In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain import hub

embedding_function = OpenAIEmbeddings()

docs = [
    Document(
        page_content="모수 비스타는 20년 이상의 요리 경력을 자랑하는 유명 셰프 알베르토 파치노가 운영하고 있습니다. 그는 지역 주민들에게 정통 이탈리아의 맛을 알리기 위해 모수 비스타를 설립했습니다.",
        metadata={"source": "owner.txt"},
    ),
    Document(
        page_content="모수 비스타는 다양한 금액에 맞는 다양한 요리를 제공합니다. 애피타이저는 8달러부터, 메인 코스는 15달러에서 30달러, 디저트는 5달러에서 10달러 사이입니다.",
        metadata={"source": "dishes.txt"},
    ),
    Document(
        page_content="모수 비스타는 월요일부터 일요일까지 영업합니다. 평일 영업시간은 오전 11시부터 오후 10시까지이며, 주말에는 오전 11시부터 오후 11시까지 연장 운영합니다.",
        metadata={"source": "restaurant_info.txt"},
    ),
    Document(
        page_content="모수 비스타는 점심 메뉴, 저녁 메뉴, 그리고 주말 특별 브런치 메뉴 등 다양한 메뉴를 제공합니다. 점심 메뉴는 가벼운 이탈리아 요리를 선보이고, 저녁 메뉴는 전통 요리와 현대적인 요리를 다양하게 제공하며, 브런치 메뉴에는 가벼운 아침 메뉴와 이탈리아 특선 요리가 모두 포함됩니다.",
        metadata={"source": "restaurant_info.txt"},
    ),
]

db = Chroma.from_documents(docs, embedding_function)
# retriever = db.as_retriever()
retriever = db.as_retriever(search_kwargs={"k": 2})

In [ ]:
retriever.invoke("언제 영업을 시작하나요?")

### RAG Prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = """다음의 컨텍스트를 토대로 답을 주세요:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = hub.pull("rlm/rag-prompt")  # use hub to pull RAG-prompt


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | ChatOpenAI()
    | StrOutputParser()
)

rag_chain.invoke("영업 시간은 언제부터 인가요?")

In [ ]:
from typing import Annotated, Literal, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain.schema import Document

llm = ChatOpenAI(model="gpt-4o-mini")
rag_chain = prompt | llm

class AgentState(TypedDict):
    messages: list[BaseMessage]
    documents: list[Document]
    on_topic: str


In [ ]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


class GradeQuestion(BaseModel):
    """질문이 모수 비스타 레스토랑과 관련이 있는지 확인하는 부울 값"""

    score: str = Field(
        description="관련되어 있다면? If yes -> 'Yes' if not -> 'No'"
    )


def question_classifier(state: AgentState):
    question = state["messages"][-1].content

    system = """사용자의 질문이 다음 주제 중 하나에 관한 것인지 판별하는 분류자 역할입니다다:

    1. 모수 비스타의 주인인 알베르토 파치노에 대한 정보입니다.
    2. 모수 비스타(레스토랑)의 음식 가격입니다.
    3. 모수 비스타(레스토랑)의 영업 시간입니다.

    질문이 다음 주제 중 하나에 관한 것이면 'Yes'로 답해주세요. 그렇지 않으면 'No'로 답해주세요. 'Yes' 또는 'No'로만 답해주세요.
    """

    grade_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", "User question: {question}"),
        ]
    )

    llm = ChatOpenAI(model="gpt-4o-mini")
    structured_llm = llm.with_structured_output(GradeQuestion)
    grader_llm = grade_prompt | structured_llm
    result = grader_llm.invoke({"question": question})
    print("RESULT", result)
    state["on_topic"] = result.score
    return state

In [ ]:
def on_topic_router(state):
    on_topic = state["on_topic"]
    if on_topic.lower() == "yes":
        return "on_topic"
    return "off_topic"


def retrieve(state):
    question = state["messages"][-1].content
    documents = retriever.invoke(question)
    state["documents"] = documents
    return state


def generate_answer(state):
    question = state["messages"][-1].content
    documents = state["documents"]
    generation = rag_chain.invoke({"context": documents, "question": question})
    state["messages"].append(generation)
    return state


def off_topic_response(state: AgentState):
    state["messages"].append(AIMessage(content="질문에 답을 할 수 없습니다!"))
    return state

In [ ]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("topic_decision", question_classifier)
workflow.add_node("off_topic_response", off_topic_response)
workflow.add_node("retrieve", retrieve)
workflow.add_node("generate_answer", generate_answer)

workflow.add_conditional_edges(
    "topic_decision",
    on_topic_router,
    {
        "on_topic": "retrieve",
        "off_topic": "off_topic_response",
    },
)

workflow.add_edge("retrieve", "generate_answer")
workflow.add_edge("generate_answer", END)
workflow.add_edge("off_topic_response", END)

workflow.set_entry_point("topic_decision")
graph = workflow.compile()

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

display(
    Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [ ]:
graph.invoke(
    input={
        "messages": [HumanMessage(content="모수 비스타 레스토랑은 언제 오픈하나요?")]
    }
)

In [ ]:
graph.invoke(
    input={"messages": [HumanMessage(content="Agentic AI란 무엇인가요?")]}
)

### Retrieval with Tools

In [ ]:
from langchain.tools.retriever import create_retriever_tool
from langchain_core.tools import tool

retriever_tool = create_retriever_tool(
    retriever,
    "retriever_tool",
    "모수 비스타 레스토랑의 영업시간과 음식 가격에 대한 정보",
)


@tool
def off_topic():
    """Catch all Questions NOT related to Pricing, Opening hours of the owner of the restaurant Bella Vista"""
    return "Forbidden - 사용자에게 응답하지 않음"


tools = [retriever_tool, off_topic]

In [ ]:
from typing import Sequence, TypedDict

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI


def agent(state):
    messages = state["messages"]
    model = ChatOpenAI()
    model = model.bind_tools(tools)
    response = model.invoke(messages)
    return {"messages": [response]}


def should_continue(state) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [ ]:
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode

In [ ]:
workflow = StateGraph(AgentState)

workflow.add_node("agent", agent)

tool_node = ToolNode(tools)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", "agent")

graph = workflow.compile()

In [ ]:
display(
    Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [ ]:
graph.invoke(
    input={"messages": [HumanMessage(content="내일 날씨는 어떤가요?")]}
)

In [ ]:
graph.invoke(
    input={
        "messages": [HumanMessage(content="모수 비스타 레스토랑은 언제 오픈하나요?")]
    }
)